In [1]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings


vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey", "some people say harrison works at kensho, but actually he works at walmart", "It is not true harrison worked at kensho", "I don't actually know where harrison works", "everyone really wants to know where harrison works", "harrison likes honey", "harrison likes like minded creatures and individuals", "if two people or creatures like to eat the same thing, we can infer that they are like minded", "both harrison and bears like to eat honey", "harrison is always thinking, i wish someone besides me liked honey", "harrison said he would like anything that also likes honey"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("does harrison like bears")

'Based on the given context, it can be inferred that Harrison likes bears because both Harrison and bears like to eat honey.'